In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Conv2D, MaxPooling2D, Flatten, BatchNormalization, Input
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import seaborn as sns

tf.__version__

# Classificação de emoções

## Carregamento da base de dados

In [ ]:
tf.keras.preprocessing.image.load_img('./fer2013/train/Angry/1003.jpg')

In [ ]:
tf.keras.preprocessing.image.load_img('./fer2013/train/Happy/1.jpg')

## Base de dados de treinamento e teste

In [ ]:
gerador_treinamento = ImageDataGenerator(rescale=1. / 255, rotation_range=7, horizontal_flip=True, zoom_range=0.2)
dataset_treinamento = gerador_treinamento.flow_from_directory('./fer2013/train',
                                                              target_size=(48, 48),
                                                              batch_size=16,
                                                              class_mode='categorical',
                                                              shuffle=True)

In [ ]:
dataset_treinamento.classes

In [ ]:
dataset_treinamento.class_indices

In [ ]:
sns.countplot(x=dataset_treinamento.classes)

In [ ]:
gerador_teste = ImageDataGenerator(rescale=1. / 255)
dataset_teste = gerador_teste.flow_from_directory('./fer2013/validation',
                                                  target_size=(48, 48),
                                                  batch_size=1,
                                                  class_mode='categorical',
                                                  shuffle=False)

## Construção e treinamento da rede neural

- Based on: https://github.com/rajeevratan84/DeepLearningCV/blob/master/18.2%20Building%20an%20Emotion%20Detector%20with%20LittleVGG.ipynb
- Padding: https://www.pico.net/kb/what-is-the-difference-between-same-and-valid-padding-in-tf-nn-max-pool-of-tensorflow
- BatchNormalization: https://keras.io/api/layers/normalization_layers/batch_normalization/
- ropout: https://jmlr.org/papers/volume15/srivastava14a.old/srivastava14a.pdf

In [ ]:
# Variáveis de configuração
numero_detectores = 32
numero_classes = 7
largura, altura = 48, 48
epocas = 70

network = Sequential([
    # 1. Camada de Entrada Explícita (Nova Prática)
    Input(shape=(largura, altura, 3)),

    # Bloco 1
    Conv2D(filters=numero_detectores, kernel_size=(3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    Conv2D(filters=numero_detectores, kernel_size=(3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.2),

    # Bloco 2
    Conv2D(filters=2 * numero_detectores, kernel_size=(3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    Conv2D(filters=2 * numero_detectores, kernel_size=(3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.2),

    # Bloco 3
    Conv2D(filters=2 * 2 * numero_detectores, kernel_size=(3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    Conv2D(filters=2 * 2 * numero_detectores, kernel_size=(3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.2),

    # Bloco 4
    Conv2D(filters=2 * 2 * 2 * numero_detectores, kernel_size=(3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    Conv2D(filters=2 * 2 * 2 * numero_detectores, kernel_size=(3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.2),

    # Camadas Densas (Classificação)
    Flatten(),

    Dense(units=2 * numero_detectores, activation='relu'),
    BatchNormalization(),
    Dropout(0.2),

    Dense(units=2 * numero_detectores, activation='relu'),
    BatchNormalization(),
    Dropout(0.2),

    # Camada de Saída
    Dense(units=numero_classes, activation='softmax')
])

network.summary()

In [ ]:
network.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Remover comentário para treinar novamente (OBS demora bastante......)
# network.fit(dataset_treinamento, epochs=epocas)

In [ ]:
network_loaded = tf.keras.models.load_model('network_emotions.keras')
network_loaded.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])
network_loaded.summary()

## Avaliação da Rede Neural

In [ ]:
network_loaded.evaluate(dataset_teste)

In [ ]:
dataset_teste.class_indices

In [ ]:
previsoes = network_loaded.predict(dataset_teste)

In [ ]:
previsoes

In [ ]:
previsoes = np.argmax(previsoes, axis=1)
previsoes

In [ ]:
dataset_teste.classes

In [ ]:
accuracy_score(dataset_teste.classes, previsoes)

In [ ]:
cm = confusion_matrix(dataset_teste.classes, previsoes)
sns.heatmap(cm, annot=True)

In [ ]:
print(classification_report(dataset_teste.classes, previsoes))


## Classificação de uma única imagem

In [ ]:
imagem = cv2.imread('fer2013/gabriel.png')
plt.imshow(cv2.cvtColor(imagem, cv2.COLOR_BGR2RGB))
plt.axis('off')
plt.show()

In [ ]:
imagem.shape

In [ ]:
# 1. Carrega o classificador (já está no seu código)
detector_face = cv2.CascadeClassifier('./haarcascade_frontalface_default.xml')

# 2. Cria uma cópia da imagem original (para desenhar nela depois)
imagem_original = imagem.copy()

# 3. --- O PASSO NOVO: CONVERTER PARA CINZA ---
# Converte de BGR (padrão OpenCV) para Grayscale
imagem_cinza = cv2.cvtColor(imagem_original, cv2.COLOR_BGR2GRAY)

# 4. Executa a detecção na imagem CINZA
# (Isso é mais rápido e preciso para esse algoritmo)
deteccoes = detector_face.detectMultiScale(imagem_cinza, scaleFactor=1.1, minNeighbors=5)


In [ ]:
deteccoes

In [ ]:
roi = imagem[40:40 + 128, 162:162 + 128]
plt.imshow(cv2.cvtColor(roi, cv2.COLOR_BGR2RGB))
plt.axis('off')
plt.show()

In [ ]:
roi = cv2.resize(roi, (48, 48))

In [ ]:
roi.shape

In [ ]:
roi

In [ ]:
roi = roi / 255
roi

In [ ]:
roi.shape

In [ ]:
roi = np.expand_dims(roi, axis=0)
roi.shape

In [ ]:
probs = network_loaded.predict(roi)
probs

In [ ]:
previsao = np.argmax(probs)
previsao

In [ ]:
dataset_teste.class_indices

## Classificação de múltiplas imagens

In [ ]:
imagem = cv2.imread('fer2013/faces_emotions.png')
plt.imshow(cv2.cvtColor(imagem, cv2.COLOR_BGR2RGB))
plt.axis('off')
plt.show()

In [ ]:
# 1. Carrega o classificador (já está no seu código)
detector_face = cv2.CascadeClassifier('./haarcascade_frontalface_default.xml')

# 2. Cria uma cópia da imagem original (para desenhar nela depois)
imagem_original = imagem.copy()

# 3. --- O PASSO NOVO: CONVERTER PARA CINZA ---
# Converte de BGR (padrão OpenCV) para Grayscale
imagem_cinza = cv2.cvtColor(imagem_original, cv2.COLOR_BGR2GRAY)
plt.imshow(cv2.cvtColor(imagem_cinza, cv2.COLOR_BGR2RGB))
plt.axis('off')
plt.show()

# 4. Executa a detecção na imagem CINZA
# (Isso é mais rápido e preciso para esse algoritmo)
deteccoes = detector_face.detectMultiScale(imagem_cinza, scaleFactor=1.1, minNeighbors=5)

In [ ]:
deteccoes

In [ ]:
dataset_teste.class_indices.keys()

In [ ]:
emotions = ['Angry', 'Disgust', 'Fear', 'Happy', 'Neutral', 'Sad', 'Surprise']

In [ ]:
for (x, y, w, h) in deteccoes:
    cv2.rectangle(imagem_original, (x, y), (x + w, y + h), (0, 255, 0), 1)

    roi = imagem_original[y:y + h, x:x + w]
    roi = cv2.resize(roi, (48, 48))
    roi = roi / 255
    roi = np.expand_dims(roi, axis=0)
    # print(roi.shape)

    previsao = network_loaded.predict(roi)
    # print(previsao)

    cv2.putText(imagem_original, emotions[np.argmax(previsao)], (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,255,0), 2, cv2.LINE_AA)

plt.imshow(cv2.cvtColor(imagem_original, cv2.COLOR_BGR2RGB))
plt.axis('off')
plt.show()

## Classificação de emoções em vídeos

In [129]:
cap = cv2.VideoCapture('./fer2013/emotion_test01.mp4')
conectado, video = cap.read()
print(conectado, video.shape)

True (360, 640, 3)


In [130]:
# fourcc.org
save_path = './fer2013/emotion_test01_result.avi'
fourcc = cv2.VideoWriter_fourcc(*'XVID')
fps = 24
output_video = cv2.VideoWriter(save_path, fourcc, fps, (video.shape[1], video.shape[0]))

In [131]:
while (cv2.waitKey(1) < 0):
  conectado, frame = cap.read()
  if not conectado:
    break
  deteccoes = detector_face.detectMultiScale(frame, scaleFactor=1.2,minNeighbors=5, minSize=(30,30))
  if len(deteccoes) > 0:
    for (x, y, w, h) in deteccoes:
      frame = cv2.rectangle(frame, (x, y), (x + w, y + h), (0,255,0), 2)
      roi = frame[y:y + h, x:x + w]
      roi = cv2.resize(roi, (48,48))
      roi = roi / 255
      roi = np.expand_dims(roi, axis = 0)
      previsao = network_loaded.predict(roi)

      if previsao is not None:
        resultado = np.argmax(previsao)
        cv2.putText(frame, emotions[resultado], (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,255,0), 1, cv2.LINE_AA)

  # plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
  # plt.axis('off')
  # plt.show()
  output_video.write(frame)

print('Terminou!')
output_video.release()
cv2.destroyAllWindows()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━